In [1]:
using DataFrames
using CSV

In [46]:
tornado_transactions_df = CSV.read("../data/tornado_transactions.csv", DataFrame)
tornado_traces_df = CSV.read("../data/tornado_traces.csv", DataFrame)

withdraw_transactions_df = CSV.read("../data/tornado_withdraw_df.csv", DataFrame)
deposit_transactions_df = CSV.read("../data/deposit_transactions.csv", DataFrame)

tornado_addresses = CSV.read("../data/tornadocontracts_abi.csv", DataFrame, header=["Address", "Currency", "3", "4","abi"])["Address"];

In [40]:
ENV["LINES"]=10
ENV["COLUMNS"]=10000

10000

Lets first look at the data obtained using the following querys: 

SELECT * FROM `bigquery-public-data.crypto_ethereum.transactions` WHERE `to_address` 
IN (SELECT `address` FROM `tornado_cash_transactions.tornadocontracts`) AND SUBSTR(`input`, 1, 10) = "0x21a0adb6";

SELECT * FROM `bigquery-public-data.crypto_ethereum.transactions` WHERE `to_address` 
IN (SELECT `address` FROM `tornado_cash_transactions.tornadocontracts`) AND SUBSTR(`input`, 1, 10) = "0xb214faa5";


In [117]:
lowercase("0x23773e65ed146a459791799d01336db287f25334")

"0x23773e65ed146a459791799d01336db287f25334"

In [116]:
lowercase("0x23773E65ed146A459791799d01336DB287f25334") ∈ tornado_addresses

true

In [9]:
deposit_transactions_df

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price
,String,Int64,Int64,String,String,Float64,Int64,Int64,String,Int64,Int64,Missing,Missing,Int64,String,Int64,String,Int64?,Int64?,Int64?,Int64
1,0xcf97c470a56d96625c7240d3004ae2abd9141d7ffc4383ab6f29a181e3562e8b,4,10,0xb050dec5a9010f8b77a3962369b7bc737d3ed4a5,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0.0,1200000,56000000000,0xb214faa5083d52f2c4e7a95e340a999805a8328b6546949ef4500ffbbd977845eade62bd,1677906,1048832,missing,missing,1,2020-11-02 17:47:30 UTC,11179130,0x21d86cba454fea4f7e43c68763d4cffec101b614554635a3f15d538049463d4f,missing,missing,missing,56000000000
2,0x2fadc4d54bf6e3ec5035f14c0096befe92999866309893e0eeaecdef955c0323,22,118,0x6e92bc493c6abbdd6a1b18416f003de2c873ab50,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0.0,1200000,63000000000,0xb214faa502843565a5499c26b04c7a9069f1ad2621aa1d8dce61480782b81d0c1c919489,8209080,1048832,missing,missing,1,2021-01-02 23:37:35 UTC,11578054,0xd52ababd1dbbbe50277c54473c881f47fd95e78081a349670d4988644e4d37c4,missing,missing,missing,63000000000
3,0x7f851ba1d7292ca565961073a1111f7408760540c6f94e479c72a2ae05399cf4,0,84,0x50b9d4af009b038506d4d84b035c451d1a3a20bc,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,1.0e17,800000,5000000000,0xb214faa51174c586ebae02a3b4f54a0ba6a9b7aba4d79ea9078c4a017a7ddb3d91150021,8465851,800000,missing,missing,0,2020-01-05 05:19:39 UTC,9218451,0x77e0fc60906714a54697cfd9275b622ad9d2d0271c4ec816b10fa3a09b103680,missing,missing,missing,5000000000
4,0xbd83053f8afa7777f54a4aca6b8e112fa31b888922dc5b9a9a65eb66e9a6996f,7,63,0x6c6e4816ecfa4481472ff88f32a3e00f2eaa95a1,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,1.0e17,800000,30838446643,0xb214faa527a20ba920c8ae877d67ce1ebd7420dafb3150e001eca78166fd6d66a5fd253e,6222489,800000,missing,missing,0,2020-05-27 03:30:44 UTC,10145408,0x837b3482443f027f6f045644bf002243f72304686015a2d6265676b2a2fc630b,missing,missing,missing,30838446643
5,0x7baf0a76f35c1dece97fff883aa7174454bed460b1bade05844080017170fc6c,240,171,0x8c4c44fd06f7f98f08bf6a9ca156cec9ee1f31f8,0xfd8610d20aa15b7b2e3be39b396a1bc3516c7144,0.0,800000,105000000000,0xb214faa50afc4880238aac4198bceddea0fbf85e9ad3de74dd20eae39f369b060b50f516,10723996,800000,missing,missing,0,2021-01-06 19:04:40 UTC,11602841,0x7c5f21ea2a92f5182ce8648f152b6fb3b4379096309dc2ba232ac663c4a0d1d7,missing,missing,missing,105000000000
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮


Here we have the first inconsistency. If you go to tornado cash stats page, you will find that the number of deposits is around 92.000
https://dune.xyz/poma/tornado-cash_1

Lets look at the withdrawls now:

In [12]:
withdraw_transactions_df

,Column1,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price,recipient_address
,Int64,String,Int64,Int64,String,String,Int64,Int64,Int64,String,Int64,Int64,Missing,Missing,Int64,String,Int64,String,Missing,Missing,Missing,Int64,String
1,0,0x8211d6815cdd607a1f8a0d5db08b3cd0121d6df4c6f5b52023788883e6fe7a97,0,54,0x00058bf70d45aa62a493e5fc596974edde903324,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf,0,421760,37000000000,0x21a0adb600000000000000000000000000000000000000000000000000000000000000e0262c5b75219fb59b3a6e535d04d7f4ddf7b8ffd3236263babebaf0e352717db22782f28dda1bfccf8b1c59d0595a400214a32bdad8dcd3b1753a4013a8d6555300000000000000000000000000058bf70d45aa62a493e5fc596974edde90332400000000000000000000000000058bf70d45aa62a493e5fc596974edde9033240000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000055def7d206642ea94235420978ff0ea81fd15e7d8d7f9ab2b261bf980bb1d321e78994d9d0c49408a22a7481681f7945d8be26efc23e183a412169da7af4422fb173319129d37b7db5df1dbb9f6f5db60f1e5f81ab07f4f030dac1ce83999e2cba949a94b22d1f4af9749f46306db16ea555cea7412951222f7fc5716eae5d0a358bd5d1527136137ec436cb67149a2ca998342359f4f2bc8904d1ae5b520522b4b6dbcdfec5217e30fcd60797cf601bd77b3b0e9157e2e2331408b2553a340e4c89700723498c6e30c1ca67a9a5d0e64a3013845f26299969e7c824aacf6e22ac16d06453342296443d0c7a37bea887ff98cb0888481525a0cbc761754d9d,2727540,321760,missing,missing,1,2020-07-16 04:21:43 UTC,10468347,0x97002c5d89010843864301297092eb60b3d902f1850d863717db7ec768e7896e,missing,missing,missing,37000000000,0x00058BF70d45AA62A493E5FC596974EdDe903324
2,1,0xcc8746c16a2bf379e8319b7c190676b77508f09f7a9cec13dd5f77514f446034,800,56,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,0,423207,13000000000,0x21a0adb600000000000000000000000000000000000000000000000000000000000000e00a52cba850f4ad215648a969675fe4f4b02d6ddae38a189e81396e742962928527e852fdad1aed9326a692b63973cf72c5d6b4a13a6c54599780e870a7c304c80000000000000000000000000039f22efb07a647557c7c5d17854cfd6d489ef30000000000000000000000000039f22efb07a647557c7c5d17854cfd6d489ef300000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010022b904e7729cc7af2d50170a9b57934740815876a1f96a43c832ece57ce95dae2d98b39685cb87a18d38e29c20271506d7103bc391b490a7d524c76c91ff4f9518442626bafb9d4d95c507fa0d3b4a19f34c457bb22dab65432c7c1cd90742200323a69be5b17c37f876c8c212625f955e69151f9b5f695899743a9798f11959226c0dfbd8cf0db70ac928850726479a0a741e0b26ad81fe1d4415d22ce258732c1a9adf37aa06373fea66b14dad706bd08c951d497b3ef8ca5c550f942cc726140dbbb541564c78a4daa44bd5ec69b192b8acf8df2ed2d86386b17abfbf991d1405c2269265226205195dd90d96119383d494b261c31e19b7b6de46ffcf9c1b,3032779,323207,missing,missing,1,2019-12-16 20:06:53 UTC,9117186,0x8dc6ade4626b61074d088ba600ddcc2686bdc1795681440ce9a86cc6dbdc4dac,missing,missing,missing,13000000000,0x0039F22efB07A647557C7C5d17854CFD6D489eF3
3,2,0x9bb7303af6ce69085abc3d9f4f5b7884a90023fd6e5925cb6ffed9737ebff78c,799,51,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,0,423207,13000000000,0x21a0adb600000000000000000000000000000000000000000000000000000000000000e00a52cba850f4ad215648a969675fe4f4b02d6ddae38a189e81396e742962928525ff03d81abc80888bb6e8bd553e9c47e8e765bd4a0ea9e96dc179ea0d2aad4f0000000000000000000000000039f22efb07a647557c7c5d17854cfd6d489ef30000000000000000000000000039f22efb07a647557c7c5d17854cfd6d489ef3000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [14]:
sorted_withdawals = sort!(withdraw_transactions_df, :block_timestamp, rev=true)

,Column1,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price,recipient_address
,Int64,String,Int64,Int64,String,String,Int64,Int64,Int64,String,Int64,Int64,Missing,Missing,Int64,String,Int64,String,Missing,Missing,Missing,Int64,String
1,17633,0x25aaa01bcefc2632525456e96c1048cdaeea32ddd12ad09c9d610ce3455a3b16,4,60,0xb5ddf5840506883467e854d48a2b95b62f021eda,0x169ad27a470d064dede56a2d3ff727986b15d52b,0,551039,8000000000,0x21a0adb600000000000000000000000000000000000000000000000000000000000000e00cee606abda2ee3409d2b6950b155ea3bb57e6c2c402aa5a3fae8e4e7801a4d906750e7e4e7798ce22984cc4d42e7b7100052ae3d8408b8243b9b6e2a349efbc000000000000000000000000c4ba2f8ae11a427693102e43f2d7be11ce129f54000000000000000000000000c4ba2f8ae11a427693102e43f2d7be11ce129f54000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100051a5cf4288e9073738a2bb2a9ed3c6540929d882720c29e6858b18430d4365a067a28da596538255a21d7e286d8f9a5ea7f72f1d0aefc59d7f14b7ec92dd2241c0253036ef4b4ee5f8c17b394952d62672038c9e5859fb0231292250bed4ae725ae4973134a6d72822192f45dc1534c729133deec202211a7a89207c8645a3207721cc51f115aa047be43970bfe3e1b372f33265c3029b91bf50c5bb5bee94d079e86cd567e745bd1f33f16743dbe45b89e98a3514f6791006fcdd77ddf8ab20663ccc13c23f31ad9ce3b3aa1474799d1f9b67362617fc39c2a2f3f7b9483502cada37a0f349008f237144b9bd57dfe84582049a33ef318f294c7c0329e8f00,13128914,411628,missing,missing,1,2021-06-10 22:00:48 UTC,12609363,0x26dc10ecdfed366c0ee1e09bb978f8d4ae7d39d12ef413305b8c66570e2304ca,missing,missing,missing,8000000000,0xc4Ba2f8aE11A427693102e43F2D7BE11ce129F54
2,17192,0x2c878be157f7eb268e2e052d566f6499735a85344079be8e547cd7ce07353562,0,52,0xa9b079d27e5992d04935ba661970f8089546db0c,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf,0,421772,13000000000,0x21a0adb600000000000000000000000000000000000000000000000000000000000000e020d7f99e545bc36fccd10cd4bd96f72d8dcc689149d59b73601d445b212a6470247878f0994800884cc5154dbef1b376a4107814f4667447c6427686270c51d0000000000000000000000000a9b079d27e5992d04935ba661970f8089546db0c000000000000000000000000a9b079d27e5992d04935ba661970f8089546db0c0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001002398a0ecaddb5c6a8f4d27599d61051615cc9855105c4da39916cf39bf8fc9581e2c0f61d73844447e507696563c13864619b123ad5b2df1b4ce58aa75b609f720753e400cd415abcd7894273d07a2cee6361ecfd0f4267f7af46008afec26902dbd521f24815d139daa88736fdc0d1b0d0240aa2c27cef275956b41c6c3041512479ec6d609eb83e3a3cda5317b53d9cc5fc423864a9a34dcbbf06f70273ebc29c4fa150d1567740dbb4a6ee45382f2d6c529fbffff08dab24ce0abbbb3f8ce0085f2f8a2ef2326fd71a36e490edaf33e1aea8215f79b768f2c676acb8340341849d8f1a8398b08ddb9fd8d7ccc53050e4c0354758550426bd51651dbfce617,5926371,273882,missing,missing,0,2021-06-05 08:10:20 UTC,12573327,0xf06eff8cc2a9a9e9c0104292a37e9d2cc866cf220446d8e6138356366b9b7965,missing,missing,missing,13000000000,0xA9B079D27E5992d04935Ba661970F8089546db0C
3,3305,0x12265a26081e9847549287523d17225ec514ea658e4e13177a3fcb70e67ea914,0,173,0x12902b10c44c6e1788061840a41a14b80d152379,0xa160cdab225685da1d56aa342ad8841c3b53f291,0,476209,55000000000,0x21a0adb600000000000000000000000000000000000000000000000000000000000000e000cb6b1b1ade6777d9152ef4d678a709ec5f7bb487152dcc6f0f361b048da32a285655efd38282e3030d2f75a67be5184e0e1875f1647dc33d1b384ab19c965600000000000000000000000012902b10c44c6e1788061840a41a14b80d15237900000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

So, it seems that we have withdrwals up to 2021-06-10. Again, this doesn't make sense.

As [this post](https://medium.com/@theimperialduke/ethereum-traces-not-transactions-3f0533d26aa) explain, in order to recalculate eth balances we need to use ethereum **traces** (check that the guys from dune also use eth.traces). Lets see:

Runing the same query as before but on the traces table:

SELECT * FROM `bigquery-public-data.crypto_ethereum.traces` WHERE `to_address` 
IN (SELECT `address` FROM `tornado_cash_transactions.tornadocontracts`) AND SUBSTR(`input`, 1, 10) IN ("0xb214faa5", "0x21a0adb6");

In [15]:
tornado_traces_df

,transaction_hash,transaction_index,from_address,to_address,value,input,output,trace_type,call_type,reward_type,gas,gas_used,subtraces,trace_address,error,status,block_timestamp,block_number,block_hash,trace_id
,String,Int64,String,String,Float64,String,String?,String,String,Missing,Int64,Int64?,Int64,String?,String?,Int64,String,Int64,String,String
1,0x14a3923eb7f30311240d6cbe838ac7e0639c309017395b36b130938d9894787c,48,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x03893a7c7463ae47d46bc7f091665f1893656003,0.0,0xb214faa50e50cde66fa6f1227cab1eeebf0aff07940d861fa4b66d3d0b4c85ce6c76bf7f,0x,call,call,missing,990735,912248,41,2,missing,1,2021-05-22 16:22:08 UTC,12485152,0xe6f35f8a11022e6b42818fff43b16690c7fcb88a7931960f7c22cc35104ebf4b,call_0x14a3923eb7f30311240d6cbe838ac7e0639c309017395b36b130938d9894787c_2
2,0xc2c9732f7ea93698d46bafa3dbe0f6febcf799945ce016fe779dd7faa2fdc87f,82,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x03893a7c7463ae47d46bc7f091665f1893656003,1.21151e17,0x21a0adb600000000000000000000000000000000000000000000000000000000000000e015d3d1d30de13ba53da140ef7120f92c7f2b7952a336d34d9db51097dc44bfc80cf98273476befb8881206a0a48af6c63dad87d7960d862d5ad886e615706a9e0000000000000000000000002fe8e10633eae71b503c60180c4190710a132237000000000000000000000000be4d1e137a24af091be80ae58d652279665e3a27000000000000000000000000000000000000000000000000000002f30d20eecb00000000000000000000000000000000000000000000000001ae69bce4dc680000000000000000000000000000000000000000000000000000000000000001000ca661773a0664c7314927e92525a32f0631194b1e73a2873b1bdff4b5b01da82e35c99e3602de30071f80fae1b55170f1c0ab03e505759fedcf2c60d84c53781205853b85e25db939e5f1f429a8ef5a1b58fae8cf10e5e1ef24f671a0743372020923e0bd54bb4ca70ae777397cbb959cb4968c0125512534c77c226de85e2c23e77641570d2dd8a46cd52438dcc0c91ec54471e6ab5f165bb76f9a598828870387d21cd35e5ed1ff4bdb1719577faacdb01ccbafaf9c16726a981d86def97b19808cb12a6b727b65dee94edf7984639cb2986e8a72da0245f137eeac5aff4603f36e0bd6033055c04d4e6805cbec2fc4b6724391a146f10a551921e3161f9a,0x,call,call,missing,500758,447979,4,0,missing,1,2021-10-15 17:05:13 UTC,13423878,0x47ed0032aee2cafd9acdf32317c434f07b537b8400f836e9186687122dd7548c,call_0xc2c9732f7ea93698d46bafa3dbe0f6febcf799945ce016fe779dd7faa2fdc87f_0
3,0xcb7e6a27cd0e9bf71054b61982b20d12e98885667ee61447d02f7ae1de9ce671,36,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x03893a7c7463ae47d46bc7f091665f1893656003,0.0,0xb214faa52c7747bcfca9ce0281515638387453bcc49c30f4f0331229e8540ecf2fa59c0a,0x,call,call,missing,1317149,941658,41,2,missing,1,2021-04-26 09:49:49 UTC,12315198,0x067f762a302cbced13ff6165da4d309d646e3abbc7863cc56f98434d5624f8cb,call_0xcb7e6a27cd0e9bf71054b61982b20d12e98885667ee61447d02f7ae1de9ce671_2
4,0xade7cb15e967438c42a9eeede86245195591a4d7f12f327f18231eaa3802ed93,68,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x03893a7c7463ae47d46bc7f091665f1893656003,0.0,0xb214faa5062e06017fab545360bc7c68684c339acc2c05a16cda23ba05074286ebdb0d3a,0x,call,call,missing,1038629,958810,41,2,missing,1,2021-04-15 21:06:02 UTC,12247016,0xd7ce8779f739c346e2e97e8b3a6f7e51896f3783ba99c1e9a23f84f8d8d21bce,call_0xade7cb15e967438c42a9eeede86245195591a4d7f12f327f18231eaa3802ed93_2
5,0x810b236b3524ec07e464db6da2befd0c70550c312422d605ad38caea2f1dbbe9,93,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x03893a7c7463ae47d46bc7f091665f1893656003,0.0,0xb214faa5268ebc52aa1ede130d959fc68d276bae6a13376f6ae637ab3330a433af3ef9ec,0x,call,call,missing,990760,912274,41,2,missing,1,2021-05-22 16:10:38 UTC,12485099,0xdd0e09ffa6d2b1e917f4e575c1bcd4e5ff22602b4c7862fdc93719871679eed9,call_0x810b236b3524ec07e464db6da2befd0c70550c312422d605ad38caea2f1dbbe9_2
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮


In [48]:
filter(row -> row.to_address ∉ tornado_addresses, tornado_traces_df) #Every row in this df is interacting with a tcash contract

,transaction_hash,transaction_index,from_address,to_address,value,input,output,trace_type,call_type,reward_type,gas,gas_used,subtraces,trace_address,error,status,block_timestamp,block_number,block_hash,trace_id
,String,Int64,String,String,Float64,String,String?,String,String,Missing,Int64,Int64?,Int64,String?,String?,Int64,String,Int64,String,String


In [23]:
deposits_df = filter(row -> first(row.input, 10) == "0xb214faa5", tornado_traces_df)

,transaction_hash,transaction_index,from_address,to_address,value,input,output,trace_type,call_type,reward_type,gas,gas_used,subtraces,trace_address,error,status,block_timestamp,block_number,block_hash,trace_id
,String,Int64,String,String,Float64,String,String?,String,String,Missing,Int64,Int64?,Int64,String?,String?,Int64,String,Int64,String,String
1,0x14a3923eb7f30311240d6cbe838ac7e0639c309017395b36b130938d9894787c,48,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x03893a7c7463ae47d46bc7f091665f1893656003,0.0,0xb214faa50e50cde66fa6f1227cab1eeebf0aff07940d861fa4b66d3d0b4c85ce6c76bf7f,0x,call,call,missing,990735,912248,41,2,missing,1,2021-05-22 16:22:08 UTC,12485152,0xe6f35f8a11022e6b42818fff43b16690c7fcb88a7931960f7c22cc35104ebf4b,call_0x14a3923eb7f30311240d6cbe838ac7e0639c309017395b36b130938d9894787c_2
2,0xcb7e6a27cd0e9bf71054b61982b20d12e98885667ee61447d02f7ae1de9ce671,36,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x03893a7c7463ae47d46bc7f091665f1893656003,0.0,0xb214faa52c7747bcfca9ce0281515638387453bcc49c30f4f0331229e8540ecf2fa59c0a,0x,call,call,missing,1317149,941658,41,2,missing,1,2021-04-26 09:49:49 UTC,12315198,0x067f762a302cbced13ff6165da4d309d646e3abbc7863cc56f98434d5624f8cb,call_0xcb7e6a27cd0e9bf71054b61982b20d12e98885667ee61447d02f7ae1de9ce671_2
3,0xade7cb15e967438c42a9eeede86245195591a4d7f12f327f18231eaa3802ed93,68,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x03893a7c7463ae47d46bc7f091665f1893656003,0.0,0xb214faa5062e06017fab545360bc7c68684c339acc2c05a16cda23ba05074286ebdb0d3a,0x,call,call,missing,1038629,958810,41,2,missing,1,2021-04-15 21:06:02 UTC,12247016,0xd7ce8779f739c346e2e97e8b3a6f7e51896f3783ba99c1e9a23f84f8d8d21bce,call_0xade7cb15e967438c42a9eeede86245195591a4d7f12f327f18231eaa3802ed93_2
4,0x810b236b3524ec07e464db6da2befd0c70550c312422d605ad38caea2f1dbbe9,93,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x03893a7c7463ae47d46bc7f091665f1893656003,0.0,0xb214faa5268ebc52aa1ede130d959fc68d276bae6a13376f6ae637ab3330a433af3ef9ec,0x,call,call,missing,990760,912274,41,2,missing,1,2021-05-22 16:10:38 UTC,12485099,0xdd0e09ffa6d2b1e917f4e575c1bcd4e5ff22602b4c7862fdc93719871679eed9,call_0x810b236b3524ec07e464db6da2befd0c70550c312422d605ad38caea2f1dbbe9_2
5,0xd5a46295d99ad3030cc487079bded0fd9b9e7fd56075f49b181a2cb430c9ee41,110,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x03893a7c7463ae47d46bc7f091665f1893656003,0.0,0xb214faa51f25c47c5a5cff19cd1b5ead8cb9f61756c888fb1c893cac089fd50f90afaf3d,0x,call,call,missing,1008171,904962,41,2,missing,1,2021-07-09 01:34:22 UTC,12790291,0x5965f85e68f6b3618274bdbf912151b01af0fb8bdae61f765b19f6773b4df3d8,call_0xd5a46295d99ad3030cc487079bded0fd9b9e7fd56075f49b181a2cb430c9ee41_2
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮


In [24]:
withdrawls_df = filter(row -> first(row.input, 10) == "0x21a0adb6", tornado_traces_df)

,transaction_hash,transaction_index,from_address,to_address,value,input,output,trace_type,call_type,reward_type,gas,gas_used,subtraces,trace_address,error,status,block_timestamp,block_number,block_hash,trace_id
,String,Int64,String,String,Float64,String,String?,String,String,Missing,Int64,Int64?,Int64,String?,String?,Int64,String,Int64,String,String
1,0xc2c9732f7ea93698d46bafa3dbe0f6febcf799945ce016fe779dd7faa2fdc87f,82,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x03893a7c7463ae47d46bc7f091665f1893656003,1.21151e17,0x21a0adb600000000000000000000000000000000000000000000000000000000000000e015d3d1d30de13ba53da140ef7120f92c7f2b7952a336d34d9db51097dc44bfc80cf98273476befb8881206a0a48af6c63dad87d7960d862d5ad886e615706a9e0000000000000000000000002fe8e10633eae71b503c60180c4190710a132237000000000000000000000000be4d1e137a24af091be80ae58d652279665e3a27000000000000000000000000000000000000000000000000000002f30d20eecb00000000000000000000000000000000000000000000000001ae69bce4dc680000000000000000000000000000000000000000000000000000000000000001000ca661773a0664c7314927e92525a32f0631194b1e73a2873b1bdff4b5b01da82e35c99e3602de30071f80fae1b55170f1c0ab03e505759fedcf2c60d84c53781205853b85e25db939e5f1f429a8ef5a1b58fae8cf10e5e1ef24f671a0743372020923e0bd54bb4ca70ae777397cbb959cb4968c0125512534c77c226de85e2c23e77641570d2dd8a46cd52438dcc0c91ec54471e6ab5f165bb76f9a598828870387d21cd35e5ed1ff4bdb1719577faacdb01ccbafaf9c16726a981d86def97b19808cb12a6b727b65dee94edf7984639cb2986e8a72da0245f137eeac5aff4603f36e0bd6033055c04d4e6805cbec2fc4b6724391a146f10a551921e3161f9a,0x,call,call,missing,500758,447979,4,0,missing,1,2021-10-15 17:05:13 UTC,13423878,0x47ed0032aee2cafd9acdf32317c434f07b537b8400f836e9186687122dd7548c,call_0xc2c9732f7ea93698d46bafa3dbe0f6febcf799945ce016fe779dd7faa2fdc87f_0
2,0x888a45d7f5dd7e289b6ff9091b97294a99c8102273f41f09d5ebfafd22e4a833,441,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x03893a7c7463ae47d46bc7f091665f1893656003,0.0,0x21a0adb600000000000000000000000000000000000000000000000000000000000000e015d3d1d30de13ba53da140ef7120f92c7f2b7952a336d34d9db51097dc44bfc801eee552023154e3c4cc8306bfcb55c99213fe34b52af4cae9f9beaeb7639e100000000000000000000000002fe8e10633eae71b503c60180c4190710a1322370000000000000000000000002fe8e10633eae71b503c60180c4190710a1322370000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001001f5c90d80840b3f7c919eb08d4fbb39744afa4c29bbe4b998b9354baa86426711095597044243810149e3aa21154e6e02fbaca1a86d85d8511fb2737c37fd0152b3f15980aafb7de7692a9db17f88b11e175713e387754ae97118465a1be2b580c37a205ba1b4d9fba65791e59a13187978b7d49351f731870264a1f8732f6f4207fc85c83e9f779556483dd0237369de809c9877673838ff24eee8ff549ba592648674dfe5856702a86fb8efdc7b86a8116839c80fb8e050a8cb8268c6b3d6127dba4df3935704f7185e8dfe6ba65fc88d0af2be0f587bf94ad759e6852d5222b1196b6cadfb5448b83bd8c1bf8e424cc9e5afef7f3f9373c0f93125b60be97,0x,call,call,missing,608808,375202,2,0,missing,1,2021-10-15 17:47:43 UTC,13424054,0x8bf82deaa17e3b3d8b7a2fe13b21ce0cce4992a64dad4175997ebd1d3087d3df,call_0x888a45d7f5dd7e289b6ff9091b97294a99c8102273f41f09d5ebfafd22e4a833_0
3,0xc54ba9eca305609d92fb0abc264a6cbf0d67a6413bb9c83beba66409722952c2,237,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x03893a7c7463ae47d46bc7f091665f1893656003,0.0,0x21a0adb600000000000000000000000000000000000000000000000000000000000000e015d3d1d30de13ba53da140ef7120f92c7f2b7952a336d34d9db51097dc44bfc82409c9fcc4cdcf4d831e9fd7c6e676a182920ad13640d556e9f66162d782acdd0000000000000000000000002fe8e10633eae71b503c60180c4190710a1322370000000000000000000000002fe8e10633eae71b503c60180c4190710a13223700000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010028ec4a404db08c0a2641e7ced47382999f46b748b73399a2c833fcc3a541c2c82b7ced21345c66f83d72269fc4fbb2d64326c55eec082dbb3ad

In [29]:
nrow(withdrawls_df) + nrow(deposits_df) == nrow(tornado_traces_df)

true

Now the number is getting closer, but is a little bit bigger than the actual number of deposits (92000)

In [83]:
succesful_deposit_traces = filter(row -> ismissing(row.error), deposits_df)

,transaction_hash,transaction_index,from_address,to_address,value,input,output,trace_type,call_type,reward_type,gas,gas_used,subtraces,trace_address,error,status,block_timestamp,block_number,block_hash,trace_id
,String,Int64,String,String,Float64,String,String?,String,String,Missing,Int64,Int64?,Int64,String?,String?,Int64,String,Int64,String,String
1,0x303a24be4f553debf80b304b607a5fa36c4b295009c1ecc609c053926b763abd,350,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf,1.0e19,0xb214faa52fc6a94bd2c89c381281a6f70a884af214e8c054e06f97883e9270f5ddc078a8,0x,call,call,missing,963163,884203,40,0,missing,1,2021-10-26 20:20:45 UTC,13495077,0x718b724a3091f4ff5ade57eb13e49d13044bf501415abac17f5ee37584bc81ba,call_0x303a24be4f553debf80b304b607a5fa36c4b295009c1ecc609c053926b763abd_0
2,0x9fcbd2b38c333829ae40c80ce8845cb19840e4124ce4338fd5678f7aad4cac64,142,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf,1.0e19,0xb214faa52511ec623ad513031dd73b88adf18ec79734ac68d186e26270e7b02a45e69c6f,0x,call,call,missing,963163,884203,40,0,missing,1,2021-10-26 20:11:48 UTC,13495037,0x18dece6bc1464885afbf47ad084d257a875f5cdeca561d187f6eace7c8e2d781,call_0x9fcbd2b38c333829ae40c80ce8845cb19840e4124ce4338fd5678f7aad4cac64_0
3,0x3b2e7ef271860c4bfe6b072b90e2d35f410c4931a9d3783438fce9a3630e1ff0,26,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf,1.0e19,0xb214faa52f1707a8a6a406922a71bfeea83155e0fad41a3502a7a4d278724d1590c848e4,0x,call,call,missing,968289,889411,40,0,missing,1,2021-10-26 20:02:57 UTC,13495007,0x756aeae3ab50b95ca4ccc855bffeefe335e86c952d1f574d7e94661fd59e5336,call_0x3b2e7ef271860c4bfe6b072b90e2d35f410c4931a9d3783438fce9a3630e1ff0_0
4,0xf762d80c5dc84f7bc7d73f290b49d5a50ac153c09df09206fc067b2dc2937912,149,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,1.0e18,0xb214faa51ffbb2cf942d4a3f22a9763e7eb8686bda7971b2cf9fc125baf2f36e51d0dbcd,0x,call,call,missing,958036,878995,40,0,missing,1,2021-10-26 19:54:01 UTC,13494967,0x6d799abe33f12a7fb5af1d38c44bdcc41a05d06ff9b628a45c867e666e51fdd1,call_0xf762d80c5dc84f7bc7d73f290b49d5a50ac153c09df09206fc067b2dc2937912_0
5,0x478ad937adb3adad0e5759e22c689da1bbf08ac95ee635bd9b725f107e2eb7d0,39,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x23773e65ed146a459791799d01336db287f25334,0.0,0xb214faa50642e8c910b5f6a680b766a82d933c1372d3bd28b9f83716447848a3236b0992,0x,call,call,missing,949955,867828,41,2,missing,1,2021-10-26 19:52:33 UTC,13494961,0x4218499f47b91300332ba2e0919b8c1388d8a616b1571eaa6a3486f53efc8aae,call_0x478ad937adb3adad0e5759e22c689da1bbf08ac95ee635bd9b725f107e2eb7d0_2
6,0xf8c02546dbf23a760304510a26d7df1368f71068dfb0c467cae4e7e5c6ac4ebb,142,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x23773e65ed146a459791799d01336db287f25334,0.0,0xb214faa504e7dd4a21ae7fdf73358423642629377696f7624c456823dff3004b8791c99a,0x,call,call,missing,949955,870821,41,2,missing,1,2021-10-26 19:50:35 UTC,13494956,0x1b8c9fc16f9d12b9e43a51a09e618e39a13d7bd770196b426c7c747d35911f09,call_0xf8c02546dbf23a760304510a26d7df1368f71068dfb0c467cae4e7e5c6ac4ebb_2
7,0x28dcf9304526689769e8f1e072f8aa102b705d3ced9ab34565a5041340aff386,88,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,1.0e18,0xb214faa50d5cfbcd26f46a8e35f557d3026d88ad5bcc6207fc983ec631b8df90d07548f5,0x,call,call,missing,958036,878995,40,0,missing,1,2021-10-26 19:37:59 UTC,13494898,0xa04a8fd9ab2ca7d0b0fd3693c91f0b9fc01a05f00c14182ea8eecab87a8798b0,call_0x28dcf9304526689769e8f1e072f8aa102b705d3ced9ab34565a5041340aff386_0
8,0xb9ae2e7abb2915056573cf79536bf77b6b062aeb1e08e49b8a72bd3c6dfc579c,206,0x722122df12d4e14e13ac3b6895a86e84145b6967,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,1.0e18,0xb214faa52a041b10b4ad20f987699993097c7efa3837bf0a9d0729fdb7bc06f7aae797f0,0x,call,call,missing,963163,884203,40,0,missing,1,2021-10-26 19:36:15 UTC,13494890,0xfd9d04b7c8c3fc2a17a9ecddb2dd677d0be1679dfc24ac775f1fe6ceba471d26,call_0xb9ae

In [109]:
sort!(combine(groupby(deposits_df, :from_address), nrow => :count), :count, rev=true)

,from_address,count
,String,Int64
1,0x722122df12d4e14e13ac3b6895a86e84145b6967,50933
2,0x905b63fff465b9ffbf41dea908ceb12478ec7601,22224
3,0xc62c539a7a6fa03f416f7d284fcc785517a607fb,145
4,0x730c4ae1626d85b9fbe5229ebe9b2f22fe12ca5a,141
5,0x34a17418cec67b82d08cf77a987941f99dc87c6b,115
6,0x6530e61a2cbbb92ce237e64c2ae9f5082d38dacd,83
7,0x54459d826a727393134c929ad1f4bc1e16f6f021,82
8,0x978aad63fe35f7a72573c411c3dff53608aaeac9,74
9,0xbe5e1ba665d397d5304f0f546cead0a1e44f242d,72


In [51]:
filter(row -> row.count>100, combine(groupby(succesful_deposit_traces, :from_address), nrow => :count))

,from_address,count
,String,Int64
1,0x722122df12d4e14e13ac3b6895a86e84145b6967,50743
2,0x730c4ae1626d85b9fbe5229ebe9b2f22fe12ca5a,140
3,0x905b63fff465b9ffbf41dea908ceb12478ec7601,22022
4,0xc62c539a7a6fa03f416f7d284fcc785517a607fb,145
5,0x34a17418cec67b82d08cf77a987941f99dc87c6b,115


Looking for "0x722122df12d4e14e13ac3b6895a86e84145b6967" and "0x905b63fff465b9ffbf41dea908ceb12478ec7601" in etherscan, we find that the first one is the "Tornado.Cash: Proxy" and the second one is "Tornado.Cash: Old Proxy"

In [107]:
sort!(combine(groupby(withdrawls_df, :from_address), nrow => :count), :count, rev=true)

,from_address,count
,String,Int64
1,0x722122df12d4e14e13ac3b6895a86e84145b6967,46822
2,0x905b63fff465b9ffbf41dea908ceb12478ec7601,14952
3,0x7d3bb46c78b0c4949639ce34896bfd875b97ad08,4065
4,0x2d50dbcc960bec854227f51f8074e1da28bd4a37,2689
5,0xe3e5e551a6a3d6664325c92833553b6bf83a865a,1612
6,0x41a28335c5075c81502a97cebad597f28728a815,1297
7,0x0b157ac0a65319b0a18aa46b0b0d4497e1074456,1275
8,0x0360b232285a801cbae01c984fb680ba2f465fdc,1011
9,0x49516e20b5692839f877a7e9aa62006a5d02a7b1,853


In [130]:
46822 + 14952

61774

Again the new and old proxy appears

In [56]:
filter(row -> row.from_address == "0x722122df12d4e14e13ac3b6895a86e84145b6967", deposit_transactions_df)

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price
,String,Int64,Int64,String,String,Float64,Int64,Int64,String,Int64,Int64,Missing,Missing,Int64,String,Int64,String,Int64?,Int64?,Int64?,Int64


In [57]:
filter(row -> row.from_address == "0x905b63fff465b9ffbf41dea908ceb12478ec7601", deposit_transactions_df)

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price
,String,Int64,Int64,String,String,Float64,Int64,Int64,String,Int64,Int64,Missing,Missing,Int64,String,Int64,String,Int64?,Int64?,Int64?,Int64


Okey, it seems that we dont have any of the transactions that interact with the proxy in our firts two df.
Lets see if we remove the two proxys, we can reconstruct our first two df

In [70]:
sum(sort!(combine(groupby(succesful_deposit_traces, :from_address), nrow => :count), :count, rev=true)[3:end, 2])

23950

Lets see how are the transactions related to the proxy traces

In [99]:
proxy_deposit_hashes = filter(row -> row.from_address == "0x722122df12d4e14e13ac3b6895a86e84145b6967", succesful_deposit_traces)["transaction_hash"]

50743-element Array{String,1}:
 "0x303a24be4f553debf80b304b607a5fa36c4b295009c1ecc609c053926b763abd"
 "0x9fcbd2b38c333829ae40c80ce8845cb19840e4124ce4338fd5678f7aad4cac64"
 "0x3b2e7ef271860c4bfe6b072b90e2d35f410c4931a9d3783438fce9a3630e1ff0"
 ⋮
 "0x688b2407342e2f2c07d9c909e4d440ba3890b6c907eb48e1b1b981f7bafb4cc2"
 "0x98e6acf253978cad6c075e8fc48deca74330b70b730931355cc3cb9195bb7242"

In [100]:
filter(row -> row.hash ∈ proxy_deposit_hashes[2:20], tornado_transactions_df)

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price
,String,Int64,Int64,String,String,Float64,Int64,Int64,String,Int64,Int64,Missing,Missing,Int64,String,Int64,String,Int64?,Int64?,Int64?,Int64
1,0x478ad937adb3adad0e5759e22c689da1bbf08ac95ee635bd9b725f107e2eb7d0,157,39,0x23cc917f53ed08c028a9b60067d4de1d7db77f5a,0x722122df12d4e14e13ac3b6895a86e84145b6967,0.0,1030830,174400213388,0x13d98d1300000000000000000000000023773e65ed146a459791799d01336db287f253340642e8c910b5f6a680b766a82d933c1372d3bd28b9f83716447848a3236b099200000000000000000000000000000000000000000000000000000000000000600000000000000000000000000000000000000000000000000000000000000000,4895665,955137,missing,missing,1,2021-10-26 19:52:33 UTC,13494961,0x4218499f47b91300332ba2e0919b8c1388d8a616b1571eaa6a3486f53efc8aae,174843499123,4000000000,2,174400213388
2,0x1c6ae3861b54437ea74e066bf439033001a2408c5541378b6e816bda188f857f,7,327,0x2522c014ed9fcf987bf156a23fd8f5231ff362ce,0x722122df12d4e14e13ac3b6895a86e84145b6967,1.0e18,997714,187296370285,0x13d98d1300000000000000000000000047ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936048d35f135291d22d74b83f8221f0bd065bccd800a351647b732debbcb0d3fdd00000000000000000000000000000000000000000000000000000000000000600000000000000000000000000000000000000000000000000000000000000000,14326306,947714,missing,missing,1,2021-10-26 19:34:05 UTC,13494875,0xd37e46228b25b9b87c42890055e28e3e603010c5f868a4742ece21a84d69605e,187316282526,4000000000,2,187296370285
3,0x9fcbd2b38c333829ae40c80ce8845cb19840e4124ce4338fd5678f7aad4cac64,1,142,0x30c54a178676e7bf97885816f50b4aa867180e8d,0x722122df12d4e14e13ac3b6895a86e84145b6967,1.0e19,1013338,264072643480,0x13d98d13000000000000000000000000910cbd523d972eb0a6f4cae4618ad62622b39dbf2511ec623ad513031dd73b88adf18ec79734ac68d186e26270e7b02a45e69c6f00000000000000000000000000000000000000000000000000000000000000600000000000000000000000000000000000000000000000000000000000000000,8816302,963338,missing,missing,1,2021-10-26 20:11:48 UTC,13495037,0x18dece6bc1464885afbf47ad084d257a875f5cdeca561d187f6eace7c8e2d781,283448593411,4000000000,2,264072643480
4,0x48bda7ccd17466111fe7bf82f933fa6450f20f08186635e423423d83ff28ccc5,87,417,0x247e8290147b82280de3cb9be5396000670b5755,0x722122df12d4e14e13ac3b6895a86e84145b6967,1.0e20,1013338,118749842175,0x13d98d13000000000000000000000000a160cdab225685da1d56aa342ad8841c3b53f2910fce3d2b81a88c4e9aa799f5d3853020c196618d71b0bb523265f06ad5f8470a00000000000000000000000000000000000000000000000000000000000000600000000000000000000000000000000000000000000000000000000000000000,27912697,963338,missing,missing,1,2021-10-26 19:14:39 UTC,13494793,0x1c3e7092afb606af73b633ea9aff63aa79e9a2769bb53ef7fe67ca7b82001076,142447730873,4000000000,2,118749842175
5,0xb9ae2e7abb2915056573cf79536bf77b6b062aeb1e08e49b8a72bd3c6dfc579c,8,206,0x2522c014ed9fcf987bf156a23fd8f5231ff362ce,0x722122df12d4e14e13ac3b6895a86e84145b6967,1.0e18,1013338,130777029758,0x13d98d1300000000000000000000000047ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c29362a041b10b4ad20f987699993097c7efa3837bf0a9d0729fdb7bc06f7aae797f000000000000000000000000000000000000000000000000000000000000000600000000000000000000000000000000000000000000000000000000000000000,13395398,963338,missing,missing,1,2021-10-26 19:36:15 UTC,13494890,0xfd9d04b7c8c3fc2a17a9ecddb2dd677d0be1679dfc24ac775f1fe6ceba471d26,201721580295,4000000000,2,130777029758
6,0xfc4dd6ace8c7995d519b6a81a3d62f7ee4a75046349cb8946b348be01cb80130,3,82,0x799f3032d7b15930a4680e20d2d7aee3abca0c3c,0x722122df12d4e14e13ac3b6895a86e84145b6967,1.0e18,1014154,134506924951,0x13d98d1300000000000000000000000047ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c29362bba072874d6c1a3f1b9f9b8e5bbaef2778d070353bf1fd41937510383c5e9ac000000000000000000000000000000000000000000000000000000000000006000000000000000000000000

Nice! The information about which Tcash contract to interact is codeded in the input data

Lets see how are the transactions regarding the old proxy

In [104]:
old_proxy_deposit_hashes = filter(row -> row.from_address == "0x905b63fff465b9ffbf41dea908ceb12478ec7601", succesful_deposit_traces)["transaction_hash"]

22022-element Array{String,1}:
 "0xa116538e2f64bb1d0e4ae84ad80b9eb4f5a2ee55914f0f8bca1e1028c12eecb5"
 "0xf1002481953fc280555e68fe8e19d95dce143917a3dff501f9af1b1dc40ed452"
 "0xa9717e259fd8d0d2d961731a492326ce08de3230a9fc84889cbc143ab04e90ee"
 ⋮
 "0x4fcd99ffeae678773c48edd8ef0d9ec32d435861ce237fbdbf87cdf17bcbbefd"
 "0x1613c7450852ec7f4380e75a118326bc61890439591cd081b51d80eb5dd5cd49"

In [105]:
filter(row -> row.hash ∈ old_proxy_deposit_hashes[1:20], tornado_transactions_df)

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price
,String,Int64,Int64,String,String,Float64,Int64,Int64,String,Int64,Int64,Missing,Missing,Int64,String,Int64,String,Int64?,Int64?,Int64?,Int64
1,0xeb146d2e4f228a08ecdd94fd81542262b12ced29027ee9cdc70dffe164b230ae,2192,56,0x6e43d04a2c8b8dc3a6fa9dd35711559b493543d1,0x905b63fff465b9ffbf41dea908ceb12478ec7601,1.0e18,1051264,130000000000,0x13d98d1300000000000000000000000047ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c29360c120a1d1316420580bd9edee544abc8e5fc3f05b5ea732ede23c3aea641e92800000000000000000000000000000000000000000000000000000000000000600000000000000000000000000000000000000000000000000000000000000000,4460942,1001264,missing,missing,1,2021-03-30 23:37:18 UTC,12143685,0x6782bf2757677acb65d8a78d81e4158a1ebed882526e9913969068c4d2450d6f,missing,missing,missing,130000000000
2,0x8990607dddd3258318969a73597e4231d22f9dd594aa954e5f283072917f284d,716,137,0xafd0b1131f1a12c7ba707e391fdd1ee67c34552b,0x905b63fff465b9ffbf41dea908ceb12478ec7601,1.0e18,1051264,146000000000,0x13d98d1300000000000000000000000047ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c29361072f2ae958fe4d2478cc6181adbc10be5499de9b88d3f1d5f1be9cefdafd87300000000000000000000000000000000000000000000000000000000000000600000000000000000000000000000000000000000000000000000000000000000,8390731,1001264,missing,missing,1,2021-03-30 22:58:24 UTC,12143499,0x40c23525f11cec536d25293011a3870220107dee0b7093b77796cb43d830f6b3,missing,missing,missing,146000000000
3,0xf1002481953fc280555e68fe8e19d95dce143917a3dff501f9af1b1dc40ed452,5,22,0x3a7edd36ab0f33cc9c4fe7b360b6bdca53bb6129,0x905b63fff465b9ffbf41dea908ceb12478ec7601,1.0e17,1051264,128000000000,0x13d98d1300000000000000000000000012d66f87a04a9e220743712ce6d9bb1b5616b8fc1a5403a046aee18eb7eaff0c338f99694a5d8de91ee983b1546634effbbeb62b00000000000000000000000000000000000000000000000000000000000000600000000000000000000000000000000000000000000000000000000000000000,2579653,1001264,missing,missing,1,2021-03-30 23:46:12 UTC,12143729,0x0dd27cf94f188f004787193cafd8d354b750d1e2d4a4f0311109753c9aa5f5da,missing,missing,missing,128000000000
4,0xa116538e2f64bb1d0e4ae84ad80b9eb4f5a2ee55914f0f8bca1e1028c12eecb5,4,162,0xd3b239c7c079f7efaec292bdc6542d7895a6959e,0x905b63fff465b9ffbf41dea908ceb12478ec7601,1.0e18,1051264,140000000000,0x13d98d1300000000000000000000000047ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c29360aa17842ea64196b99478e99ea2a9c442ea6332a5b571578402d8b301a2bc97e00000000000000000000000000000000000000000000000000000000000000600000000000000000000000000000000000000000000000000000000000000000,8489658,1001264,missing,missing,1,2021-03-30 23:49:35 UTC,12143749,0x7ad62a74ca142af5c56b37713999c41187de345faad093769062488e046269c6,missing,missing,missing,140000000000
5,0x26931119877034cddf8c39fb20df1e499bf1861b3711ff3c6810d4f028291614,3,29,0x3a7edd36ab0f33cc9c4fe7b360b6bdca53bb6129,0x905b63fff465b9ffbf41dea908ceb12478ec7601,1.0e17,1051264,130000000000,0x13d98d1300000000000000000000000012d66f87a04a9e220743712ce6d9bb1b5616b8fc1e8f33720892c52dd0d1aadd1b55799384cd753e5151cbdaafa6389cffcc4d9200000000000000000000000000000000000000000000000000000000000000600000000000000000000000000000000000000000000000000000000000000000,2382914,1001264,missing,missing,1,2021-03-30 23:44:54 UTC,12143719,0x2d7060ea3863a0e7094412aa0d9fa9d6d0cf208af5d46281a0fc34ff576d94ce,missing,missing,missing,130000000000
6,0x1d8da2f8759ec4ed2e6932ddb5e214e1d1c0d7aa1d223744aacf288bf3b1182e,12,49,0x3caeee72b54cd518d2ea97e95607499c3d9e4336,0x905b63fff465b9ffbf41dea908ceb12478ec7601,1.0e18,1063080,120000000000,0x13d98d1300000000000000000000000047ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c293604c60a7b6993352ad74b3765f015621d08d463edcaa4161aab0dd195e9a0c09c000000000000000000000000000000000000000000000000000000000000006000000000000000000000000000

In [115]:
proxy_deposit_transactions = filter(row -> first(row.input, 10) == "0x13d98d13" && row.value == 0.0, tornado_transactions_df)

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price
,String,Int64,Int64,String,String,Float64,Int64,Int64,String,Int64,Int64,Missing,Missing,Int64,String,Int64,String,Int64?,Int64?,Int64?,Int64
1,0xc0fbe9da9cad77f410cdf436a57754e475e124189ecd69a3dba37261dd7efe2d,113,100,0xa5294016f13fb0015c1d22bd521eb65a123d34dc,0x722122df12d4e14e13ac3b6895a86e84145b6967,0.0,1031828,24000000000,0x13d98d1300000000000000000000000023773e65ed146a459791799d01336db287f2533420a0defd179aec6cd23f8d01a1c490b15823d6749ab5ff947c920ecd89105fa400000000000000000000000000000000000000000000000000000000000000600000000000000000000000000000000000000000000000000000000000000000,6031107,944128,missing,missing,1,2021-07-17 19:18:27 UTC,12846082,0x0db97b6851d8bb165e4b1dc98e3e6a3bfbe08f7bc79d3814cc151699d47407d7,missing,missing,missing,24000000000
2,0xf134cf9457448af20c643451389c38a206c34eda188860a7df3b0d70dfc356a5,147,84,0x941a9e3b91e1cc015702b897c512d265fae88a9c,0x722122df12d4e14e13ac3b6895a86e84145b6967,0.0,1031828,8000000000,0x13d98d1300000000000000000000000023773e65ed146a459791799d01336db287f253341e5ac1ceb652fa7f951d5cc14cc0658f297bc295317d36e12fe441da8f74431a00000000000000000000000000000000000000000000000000000000000000600000000000000000000000000000000000000000000000000000000000000000,10258686,944128,missing,missing,1,2021-07-04 00:00:35 UTC,12757716,0xd2f57ed3b20d232ad152c3fcac7d5e8bfcafe21e6c674fd6a1b7450f9d7cf548,missing,missing,missing,8000000000
3,0x00a5e51e0c10a36cd44c9ee1ba464a70b70f87ce15119c7446e67f1af1e1c8d2,1,64,0xcf8d8e7378abdd864f49cd74071bcb35582f6315,0x722122df12d4e14e13ac3b6895a86e84145b6967,0.0,1031828,10000000000,0x13d98d1300000000000000000000000007687e702b410fa43f4cb4af7fa097918ffd273003cd53f8e01e4fe0af02ee752cce8d5b5c028650bbacbd93e25362c3f4756ba900000000000000000000000000000000000000000000000000000000000000600000000000000000000000000000000000000000000000000000000000000000,7154618,944128,missing,missing,1,2021-07-05 13:14:17 UTC,12767706,0x88c7f8729ce598a919b6eafbb9f262914086239391471262bc69c1f4b99e0414,missing,missing,missing,10000000000
4,0x7ad162c4476b30ab5a1769ab8eb72df808d7bb05b656884fa476997f09a76dd6,86,53,0x814d8daef1ddfbc8ca9bc16f00bb5b46d2eea388,0x722122df12d4e14e13ac3b6895a86e84145b6967,0.0,1031828,14000000000,0x13d98d1300000000000000000000000023773e65ed146a459791799d01336db287f253342abbdc4bc7db7b1fdf7c81bce220461bb5a2dcbbad929222bef36e93538e5cf000000000000000000000000000000000000000000000000000000000000000600000000000000000000000000000000000000000000000000000000000000000,6545414,944128,missing,missing,1,2021-07-18 19:40:07 UTC,12852539,0x5c52279bdf00250071b9fa5a9a30249df743eb6fb39afb0604414547f34890dd,missing,missing,missing,14000000000
5,0x325fec3c8aaa5e8a0c48946363df7fb226e13ab190b04d7e0dbc1f3627f2b954,12,206,0xb493f2cc5f0d781affe7cdbe60500e2a8d3d7b90,0x722122df12d4e14e13ac3b6895a86e84145b6967,0.0,1031868,41000000000,0x13d98d1300000000000000000000000023773e65ed146a459791799d01336db287f253342927c0c918100eb57d5e1c8c8380ea03d0d2173bca947adeaf00ad8d7e6a123d00000000000000000000000000000000000000000000000000000000000000600000000000000000000000000000000000000000000000000000000000000000,14007947,954368,missing,missing,1,2021-08-12 19:14:31 UTC,13012199,0xa1bc18336718139548a3452417f8f2706f332fb965e3c51d87f89f6b07f42247,missing,missing,missing,41000000000
6,0x23047e2397435fa0df90b0d6be35b8181f04ea065dc0ffa25ed5c8477d223ff1,51,144,0x94e102b8f08a5a2b584e7911e9fc60bc8c519f70,0x722122df12d4e14e13ac3b6895a86e84145b6967,0.0,1021877,28000000000,0x13d98d1300000000000000000000000023773e65ed146a459791799d01336db287f2533424b091ff2c8eddeb023903cb631ceb5a1d1012a0c99e0aa76e0674cb5747dbf000000000000000000000000000000000000000000000000000000000000000600000000000000000000000000000000000000000000000000000000000

In [120]:
succesful_deposit_traces_day = filter(row -> row.to_address == "0x23773e65ed146a459791799d01336db287f25334" && row.value != 0, succesful_deposit_traces)

,transaction_hash,transaction_index,from_address,to_address,value,input,output,trace_type,call_type,reward_type,gas,gas_used,subtraces,trace_address,error,status,block_timestamp,block_number,block_hash,trace_id
,String,Int64,String,String,Float64,String,String?,String,String,Missing,Int64,Int64?,Int64,String?,String?,Int64,String,Int64,String,String


It seems that every transaction that calls the dai100k (every os the dai contracts) contract has a value of cero

In [125]:
succesful_deposit_traces_day = filter(row -> row.to_address == "0xd4b88df4d29f5cedd6857912842cff3b20c8cfa3" && row.value != 0, succesful_deposit_traces)

,transaction_hash,transaction_index,from_address,to_address,value,input,output,trace_type,call_type,reward_type,gas,gas_used,subtraces,trace_address,error,status,block_timestamp,block_number,block_hash,trace_id
,String,Int64,String,String,Float64,String,String?,String,String,Missing,Int64,Int64?,Int64,String?,String?,Int64,String,Int64,String,String


Lets see what happen with the withdrawls using the proxy

In [127]:
proxy_withdrawls_hashes = filter(row -> row.from_address == "0x722122df12d4e14e13ac3b6895a86e84145b6967", withdrawls_df)["transaction_hash"]

46822-element Array{String,1}:
 "0xc2c9732f7ea93698d46bafa3dbe0f6febcf799945ce016fe779dd7faa2fdc87f"
 "0x888a45d7f5dd7e289b6ff9091b97294a99c8102273f41f09d5ebfafd22e4a833"
 "0xc54ba9eca305609d92fb0abc264a6cbf0d67a6413bb9c83beba66409722952c2"
 ⋮
 "0x0f7694c15d7459f087c03f154664d2b9143974ab11c4b51d0847d993d70bd8ab"
 "0x4086ceb0bed689edeeb6630ee0d9f267adef8e3e6cc32257f97cead4dab2a8b1"

In [128]:
filter(row -> row.hash ∈ proxy_withdrawls_hashes[1:20], tornado_transactions_df)

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price
,String,Int64,Int64,String,String,Float64,Int64,Int64,String,Int64,Int64,Missing,Missing,Int64,String,Int64,String,Int64?,Int64?,Int64?,Int64
1,0x7639f23e09f70f766d10085582c510878c7aa6765fd7a36058dd5de33e9898f9,1763,63,0x6a77500a141fede525e57e422d65d67a32f2d8fe,0x722122df12d4e14e13ac3b6895a86e84145b6967,0.0,655624,101000000000,0xb438689f00000000000000000000000003893a7c7463ae47d46bc7f091665f189365600300000000000000000000000000000000000000000000000000000000000001000ba62c52144ea8db7be89f4539fb947461a9c088d4322b312ccc6be243568a610a374f620bac72518dda7c489fdbb7839b74b933e291a9e5c1340e4df873b5240000000000000000000000006a77500a141fede525e57e422d65d67a32f2d8fe0000000000000000000000006a77500a141fede525e57e422d65d67a32f2d8fe00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010020664894ee4e8367219c16e686f54fd88a259ddc0ce435daa160334dbc9a82520dc17a888de690d7087d19760e4045ec4d4a6e6a4b6da5a2608eee0e57171723038cd033996be06fff56cc6bcaefc01ee288497e1df308433b9e116403527cf716c355a4c8e249d9b879708963702c93e5384f581682ebad8be50eb2ce93c098101240cbc95bdf8d2cc8bbdf50e7aa74c8dd9068099dc862da6d49a381ad739625160e35bc0dbd1bd0dcea863d48fecf92088a3b12fcaf46fab3864ec1850f9d0717a290ebc559c2b3290330f5c2b69c71b8b257aabcd2ba856cd113292bacc0159f3a6f6b3ce659dcd489a752e67977e8bbc5707fd5a2a87e7a4008796f7094,3773972,449019,missing,missing,1,2021-04-16 00:29:51 UTC,12247941,0x680bd269e57413d499aa90956c1c8ff456f07b1a09fe4b8c7d58729cdd37b858,missing,missing,missing,101000000000
2,0x1de0aa750fa80431e735a2ad4354570e028d6239d3e37483b147bfa8aafed346,5738,97,0x2fe8e10633eae71b503c60180c4190710a132237,0x722122df12d4e14e13ac3b6895a86e84145b6967,0.0,652921,150552114813,0xb438689f00000000000000000000000003893a7c7463ae47d46bc7f091665f1893656003000000000000000000000000000000000000000000000000000000000000010015d3d1d30de13ba53da140ef7120f92c7f2b7952a336d34d9db51097dc44bfc80ec9e08ebeb7fc2b1b5db455c617cd5fbcf6ca9a9f71012537b62a8f45323c860000000000000000000000002fe8e10633eae71b503c60180c4190710a1322370000000000000000000000002fe8e10633eae71b503c60180c4190710a1322370000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000f7d2bff156b136a00d80ac11e13bf516c1be51ea74d66e50edb93f01d244aeb301b3e865e1dca6feff582b19be059aa3715c8285629940a33c82d8261db710e0473d97a350940990f5807c43060b8e87c8a679ed34abdc904c3c921fd6298562e68c15b4f7d31b4a2ccf95cbf1f478068f9d5602f013f6cf5ff0324a322e2e400d9fce9cd04ca3bce2ab19b38d49ea093d7ea0836e30d9857e2081dae0e5d7607d3d97b9d214302d6b05b63d10a98803a2d53b323749a6eb1232ee87ff35a5b08c36d449817c8ce94e225e91d88cb5bc79176d2e878bb27cf0eb03f02821e550b3d65ed1226e021161d9dba0e0187f4373e201458d98bdc4c55aa7bfa6eb75a,5605470,446316,missing,missing,1,2021-10-15 17:43:43 UTC,13424032,0xb4f676dd3e41f5babeee54908f9c61e13510fb4ffd0f1b36d1155f7da41c6123,178128826899,4000000000,2,150552114813
3,0xc54ba9eca305609d92fb0abc264a6cbf0d67a6413bb9c83beba66409722952c2,5729,237,0x2fe8e10633eae71b503c60180c4190710a132237,0x722122df12d4e14e13ac3b6895a86e84145b6967,0.0,652933,148067391429,0xb438689f00000000000000000000000003893a7c7463ae47d46bc7f091665f1893656003000000000000000000000000000000000000000000000000000000000000010015d3d1d30de13ba53da140ef7120f92c7f2b7952a336d34d9db51097dc44bfc82409c9fcc4cdcf4d831e9fd7c6e676a182920ad13640d556e9f66162d782acdd0000000000000000000000002fe8e10633eae71b503c60180c4190710a1322370000000000000000000000002fe8e10633eae71b503c60180c4190710a1322370000000000000000000000000000000000000000000000000000000000

In [133]:
proxy_withdrawl_transactions = filter(row -> first(row.input, 10) == "0xb438689f" && row.to_address ∉ ("0x722122df12d4e14e13ac3b6895a86e84145b6967", "0x905b63fff465b9ffbf41dea908ceb12478ec7601") , tornado_transactions_df)

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price
,String,Int64,Int64,String,String,Float64,Int64,Int64,String,Int64,Int64,Missing,Missing,Int64,String,Int64,String,Int64?,Int64?,Int64?,Int64
1,0x2237c22e71d094a29b4967afe7f9c95e2a0328d932d6413f80cc7b88bcfc77c2,1,6,0x20c62682cbe1eea82ef7a985b3805f5c360813ac,0x34410f5b9a1aaad11961b50a62c1d2913cd3a588,0.0,500000,0,0xb438689f00000000000000000000000047ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c29360000000000000000000000000000000000000000000000000000000000000100216105ad90bb0f93a30ed938628e0c838e6439bde343e94b5f9c87b0f6855eb711810579543ee4ed1ac8bf116df3e0e49a127c135c19c678fc4fcb9adba0a84400000000000000000000000003ebd0748aa4d1457cf479cce56309641e0a98f500000000000000000000000034410f5b9a1aaad11961b50a62c1d2913cd3a588000000000000000000000000000000000000000000000000008e1bc9bf040000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001002d51f312c5f7e49160f29b499fa12590021810b7bf83dbbcc55c579060be4aaa180c5c5ed5d431cc3a2ec5e5597c977aa3fa515adcfe2a285becb14392a9bb6c197f30a3b993553c9a05ba569fc17148ddcb6b9fe1b1306e6e738f48321d0b98223e854698c7ed345f066ca358ff4e9f27fd2b9a64650781c6d4f658c96b5e0f068f4b3c71c6d2ffb17099747edc1773cebf874294744f6aa913b5081f663fb12c0c443060fdec6ab70edae319e4d706ae274f00a7d076901338fb6f1264b49717e89e7f4d132e9653f1e103134311f51c46b208abbc74d098a2f56ed61b4c762734226bbdf85a0644890cb9814a48a531a8dbf110eca905e96281a4b7ab0cc0,947778,389769,missing,missing,1,2021-04-27 15:11:48 UTC,12323153,0x4751d27982b19a4b93ca54b3867f19e032e165d1492ffefc53a62d2ca00b6770,missing,missing,missing,0
2,0x08a19adc2ad814f6d3a76a8bcf52e5a609fff5c39fd376438525bf62a36fd24e,0,100,0xead23b83f5bfce6795724d559a896f452c9d3113,0xfcf6a5e26cd0317a446e5ef1872b1374a4e517d7,0.0,414563,44000000000,0xb438689f00000000000000000000000047ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c293600000000000000000000000000000000000000000000000000000000000001001ba048389c0fffab399d4f02fb99768851beae07c5b9f4a8701989e2bb68da1521ad129e1d68fecc6fc6cac844bedda2413fc4c49a763f250258f118179375c7000000000000000000000000a665cb7e25a15fbc558bc4adec07c2b592dc693f000000000000000000000000ead23b83f5bfce6795724d559a896f452c9d3113000000000000000000000000000000000000000000000000005fb5c98a3fc00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100051d3a33c2389dcace74aa155ddc7387f9c83b76a627a8cfeabbc48588c48e9c0c539ae146fc42893770afc401f91af525b850b53d764a962e138de4b40632661f4ce1724f82c127c45ef0b358f1f95226537502d98022fd7a37b15db54e448518ebb7547d381560f22abc7aeab681669690090a85d29b357a3b754befb281d01b394cee4b87fe48e84e729181633e70c40a0212d948b97ed47b074ac6933a50272039d4acee34f945bf755b531559d8c008c7ef0984c850d3abb0fa46af038b1a8ec2e0ebcf65aeefc28dcaf481ab81bca9f087bda2d1e68afa25d98bf7a45513d994ace97a1f49ae4a939d4d879effed5e907ff5043fd038df748faa8a3f0b,7720259,376275,missing,missing,1,2020-11-30 22:02:17 UTC,11362749,0xe42e067f6799edd609a9b57f32d3ee4af3070103f6f4ffead02ad0b619a9d55c,missing,missing,missing,44000000000
